In [60]:
import numpy as np
import math


def position_calc(k1, k2, x0, y0):
    """
    Makes necessary calculations
    
    Arguments:
        k1 - rotation around Z axis angle. from -1 to 1
        k2 - incline  angle.               from  0 t0 1
        x0 - x coordinate of the point on the screen, pixels
        y0 - y coordinate of the point on the screen, pixels
        
    """   
    
    pi  = np.pi
    cos = math.cos
    sin = math.sin


    ###__constants__####

    h0 =  45.959                    #height of the neck joint, cm
    h2 =  5.071                     #position of the photomatrix relative to the neck joint along the Y axis, cm
    h3 =  6.364                     #position of the photomatrix relative to the neck joint along the Z axis, cm
    f  =  600                       #photomatrix-lens distanse, px


    e1 =  np.array([0, 0, 1])              #Z axis vector 
    e2 =  np.array([1, 0, 0])              #X axis vector
    H  =  h0 * e1                          #neck joint vector
    L0 =  np.array([0, h2, h3])            #position of the lens reletive to neck joint vector
    q0 = -np.array([x0, f, y0])            #position of the point on the photomatrix vector reletive to lens
    Q0 =  L0 + q0                          #position of the point on the photomatrix vector reletive to neck joint



    def rotation_matrix(axis, theta):
        """
        Return the rotation matrix associated with counterclockwise rotation about
        the given axis by theta radians.
        """
        axis = np.asarray(axis)
        axis = axis / math.sqrt(np.dot(axis, axis))
        a = math.cos(theta / 2.0)
        b, c, d = -axis * math.sin(theta / 2.0)
        aa, bb, cc, dd = a * a, b * b, c * c, d * d
        bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
        return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                         [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                         [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])


    a1 =  pi/2 * k1                  #rotation around Z axis angle
    a2 = -pi/2 * k2                  #incline  angle

    R1 = rotation_matrix(e1, a1)     #rotaion around Z axis matrix
    R2 = rotation_matrix(e2, a2)     #incline up-down matrix
    R  = np.matmul(R1, R2)           #full rotation matrix


    Q = R.dot(Q0) + H                #position of the point on the screen reletive to floor vector
    q = R.dot(q0)                    #ligth ray vector
    
    return [Q, q]

def point_position(k1, k2, x0, y0):
    """
    Takes same args as calc_position
    Returns the position of the point on the floor reletive to robot
    """
    calc = position_calc(k1, k2, x0, y0)   #takes calculations from position_calc
    
    Q = calc[0]                            #position of the point on the screen reletive to floor vector
    q = calc[1]                            #ligth ray vector
    
    A = Q - q * Q[2] / q[2]                #position of the point on the floor reletive to robot
    
    return A

def ball_position(k1, k2, x0, y0):
    """
    Takes same args as calc_position
    Returns the position of the ball on the floor reletive to robot
    """    
    r = 5                                  #radius of the ball
    calc = position_calc(k1, k2, x0, y0)   #takes calculations from position_calc
    
    Q = calc[0]                            #position of the point on the screen reletive to floor vector
    q = calc[1]                            #ligth ray vector
    
    
    B = Q - q * (Q[2] - r) / q[2]          #position of the ball on the floor reletive to robot
    B[2] = 0                               #on the floor z = 0
    
    return B
    

In [61]:
x0 =  1        #x coordinate of the point on the screen, pixels
y0 = -1.3      #y coordinate of the point on the screen, pixels
k1 = -0.1      #incline  angle.               from  0 t0 1
k2 =  0.2      #rotation around Z axis angle. from -1 to 1

A = point_position(k1, k2, x0, y0)
B = ball_position(k1, k2, x0, y0)


In [62]:
print(A)
print(B)

[ 25.43976558 158.85779236   0.        ]
[ 23.02365754 143.77433145   0.        ]


1228800